# Readme

Computes the intial states $y_0$ of for all panels.

This is a semi automatic procedure, where the network is simulated and a point is picked that resembles the initial point in the figure below.

For some panels this requires some fine-tuning of solver because the solutions are not numerically stable.

Note: The figure you might want to load it from Prinz et al. 2004 Figure 3. It's not mine, therefore it is not part of this repo.

# Imports

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from sys import path as sys_path
from os.path import abspath as os_path_abspath
sys_path.append(os_path_abspath('..'))
import addpaths

In [ ]:
import data_utils
import plot_utils as pltu

# Solver functions

In [ ]:
import stg_model
from data_generator_STG import data_generator_STG

def get_panel_generator(panel, tmax):
    """Get generator for panel"""
    
    network = stg_model.stg_model(n_neurons=3, g_params=panel, y0from1n=True)
    
    gen = data_generator_STG(
        y0=network.y0, t0=0.0, tmax=tmax*1e3, t_eval_adaptive=None,
        vidx=None, yidxs=np.arange(46), model=network, n_samples=1,
        gen_det_sols=False, gen_acc_sols=False, 
        base_folder='_data/intial_states'
    )
    
    gen.update_subfoldername(panel=panel)
    
    return gen

In [ ]:
def get_y_at_t(data, t):
    """Get y at t"""
    
    ts = data.ts
    ys = data.ys
    tidx = np.argmin(np.abs(ts - t))
    return ys[tidx,:]

In [ ]:
def plot_data(data, plot_t_rng, y0_t=None):
    """Plot data and show y0 that is currently picked"""
    
    y0_t *= 1e3
    
    ts = data.ts
    ys = data.ys

    vss = ys[:,0], ys[:,13], ys[:,26]

    plot_idxs = (ts >= plot_t_rng[0]*1e3) & (ts <= plot_t_rng[1]*1e3)
    y0 = get_y_at_t(data, y0_t) if y0_t is not None else None

    fig, axs = pltu.subplots(3, 2, squeeze=True, sharex='row', sharey=False)
    for i, vs in enumerate(vss):
        axs[0,i].set_title(f'n{i+1}: Zoom')
        axs[0,i].plot(ts[plot_idxs], vs[plot_idxs])
        axs[0,i].axvline(y0_t, c='r')
        axs[0,i].axhline(y0[[0,13,26][i]], c='r')
        axs[0,i].set(xlabel='Time', ylabel='v(t)')
        
        axs[1,i].set_title(f'n{i+1}: Full')
        axs[1,i].plot(ts, vs)
        axs[1,i].axvline(y0_t, c='r')
        axs[1,i].axhline(y0[[0,13,26][i]], c='r')
        axs[1,i].set(xlabel='Time (ms)', ylabel='v(t) (mV)')
        
    plt.tight_layout()

In [ ]:
def show_panel_data(
        panel, tmax=0.1, method='RKDP', adaptive=True, step_param=1e-5,
        plot_t_rng=(0,np.inf), y0_t=None, save2dict=False
    ): 
    """load or generate data for panel. Plot data. May save eq state to dict: n3_panel2y0"""
    
    gen = get_panel_generator(panel=panel, tmax=tmax)
    gen.gen_and_save_data(
        method=method, adaptive=adaptive, step_param=step_param,
        pert_method=None, plot=False, overwrite=False, allowgenerror=False
    )
    data = gen.load_data_and_check(method=method, adaptive=adaptive, step_param=step_param, pert_method=None)
    
    plot_data(data, plot_t_rng=plot_t_rng, y0_t=y0_t)
    
    if save2dict: n3_panel2y0[panel] = get_y_at_t(data, y0_t*1e3)

## Test

Pick a differetnt "y0_t" or "plot_t_rng"

In [ ]:
y0_t = 1.95
plot_t_rng = (1.9, 2.1)
assert y0_t > plot_t_rng[0] and y0_t < plot_t_rng[1], "Pick a different time window"

show_panel_data(panel='a', tmax=2, y0_t=y0_t, plot_t_rng=plot_t_rng, save2dict=False)

# Generate y0s

This is a semi manual procedure.
- Specify a panel and run the model.
- Plot the data.
- Find a good t0, such that the output looks as in the paper figure.
- Save y0 at this t to the dict.
- Repeat for other panels.
- Save y0_dict to file.

In [ ]:
import os

y0_filename = '../../models/stg_n3_panel2y0.pkl'

if os.path.isfile(y0_filename):
    print('Loaded file.')
    n3_panel2y0 = data_utils.load_var(y0_filename)
    assert isinstance(n3_panel2y0, dict)
else:
    print('Created new dict.')
    n3_panel2y0 = {}

## abcde

In [ ]:
show_panel_data(panel='a', tmax=3, y0_t=1.945, plot_t_rng=(1.9, 2.1), save2dict=True)

In [ ]:
show_panel_data(panel='b', tmax=6, y0_t=1.9, plot_t_rng=(1.85, 1.95), save2dict=True)

In [ ]:
show_panel_data(panel='c', tmax=6, y0_t=3, plot_t_rng=(2.9, 3.1), save2dict=True)

In [ ]:
show_panel_data(panel='d', tmax=10, y0_t=1.2, plot_t_rng=(1.1, 1.3), save2dict=True)

In [ ]:
show_panel_data(panel='e', tmax=6, y0_t=4.5, plot_t_rng=(4.0, 4.9), save2dict=True)

Load the Fig. 3 from Prinz et al. for easier comparison:
<img src="_background/n3_traces_top_row_small.png">

## fghij

In [ ]:
show_panel_data(panel='f', tmax=6, y0_t=4, plot_t_rng=(3.9, 4.2), save2dict=True)

In [ ]:
show_panel_data(panel='g', tmax=6, y0_t=3.01, plot_t_rng=(2.9, 3.1), save2dict=True)

In [ ]:
show_panel_data(panel='h', tmax=6, y0_t=4.29, plot_t_rng=(4.1, 4.5), save2dict=True)

In [ ]:
show_panel_data(panel='i', tmax=6, y0_t=0.2, plot_t_rng=(0.1, 0.3), save2dict=True)

In [ ]:
show_panel_data(panel='j', tmax=6, y0_t=2.45, plot_t_rng=(2.2, 2.6), save2dict=True)

Load the Fig. 3 from Prinz et al. for easier comparison:
<img src="_background/n3_traces_bottom_row_small.png">

# Save

In [ ]:
data_utils.save_var(n3_panel2y0, y0_filename)